## Description:
This task calculates estimated service line costs in outpatient claims by applying provider-specific cost-to-charge ratios (CCRs). CCRs are simulated at the provider level across revenue codes and merged into the claim data. For each claim, line-level costs are computed based on matching revenue codes. The result includes both a claim-level dataset with estimated costs for each service line and a line-level dataset for detailed analysis.

In [22]:
# Here’s the code to simulate the provider-level CCR file:

import pandas as pd
import numpy as np

# Provider IDs
prov_ids = np.arange(100, 401)

# Revenue centers
rev_center_pool = [
    "0450", "0456", "0510", "0360", "0300",
    "0270", "0320", "0250", "0636", "0260",
    "0420", "0430", "0440", "0460", "0480",
    "0490", "0520", "0730", "0610", "0910"
]

# Build provider-level CCRs
data = []
for prov in prov_ids:
    row = {"prov_id": prov}
    for rvcd in rev_center_pool:
        row[f"ccr_{rvcd}"] = round(np.random.uniform(0.1, 1.0), 4)
    data.append(row)

# Create DataFrame
df_prov_ccr = pd.DataFrame(data)

In [23]:
df_prov_ccr.head(5)

,prov_id,ccr_0450,ccr_0456,ccr_0510,ccr_0360,ccr_0300,ccr_0270,ccr_0320,ccr_0250,ccr_0636,...,ccr_0420,ccr_0430,ccr_0440,ccr_0460,ccr_0480,ccr_0490,ccr_0520,ccr_0730,ccr_0610,ccr_0910
0,100,0.5082,0.4090,0.5132,0.1652,0.4449,0.9244,0.7411,0.4938,0.2686,...,0.7515,0.2906,0.1546,0.4964,0.3508,0.3685,0.6270,0.4714,0.2706,0.8103
1,101,0.5238,0.6896,0.5308,0.7966,0.1601,0.2810,0.9547,0.7088,0.1359,...,0.1772,0.4842,0.6144,0.3316,0.9750,0.6809,0.5171,0.4344,0.7319,0.9855
2,102,0.6605,0.4499,0.2423,0.6267,0.3294,0.7958,0.4354,0.6937,0.4191,...,0.4015,0.1598,0.2944,0.3435,0.1688,0.6977,0.2574,0.2239,0.1147,0.1124
3,103,0.6762,0.4639,0.3002,0.9505,0.2834,0.5583,0.4750,0.6915,0.1970,...,0.3450,0.6165,0.7842,0.9576,0.3117,0.2547,0.2281,0.4183,0.5313,0.8846
4,104,0.4972,0.8914,0.8484,0.9369,0.4936,0.6393,0.7886,0.3227,0.3599,...,0.5989,0.5131,0.4191,0.3958,0.8090,0.6329,0.8175,0.8402,0.1442,0.4141


In [43]:
# readin the input, opps claims
df_opps_claims = pd.read_csv(r"J:/Python/Learning/Data/opps_claims_simulated.csv")
df_opps_claims.head()

,claim_id,bene_id,prov_id,line_ct,line1_hcpcs,line1_rev,line1_chrg,line1_units,line1_revdt,line2_hcpcs,...,line4_hcpcs,line4_rev,line4_chrg,line4_units,line4_revdt,line5_hcpcs,line5_rev,line5_chrg,line5_units,line5_revdt
0,0,2102,303,5,J2505,420,1180.55,5,2025-01-15,J2778,...,J9206,456.0,1096.90,6.0,2025-01-26,G0463,450.0,491.15,6.0,2025-01-04
1,1,2088,300,3,J3489,490,387.52,3,2025-01-19,J0885,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2166,301,4,J0881,730,608.85,5,2025-01-26,G0463,...,C8900,730.0,1010.66,2.0,2025-02-01,NaN,NaN,NaN,NaN,NaN
3,3,2133,301,4,J1459,430,1412.27,4,2025-01-15,J0881,...,J1569,636.0,479.60,1.0,2025-02-01,NaN,NaN,NaN,NaN,NaN
4,4,2088,300,1,J0885,250,58.01,3,2025-01-16,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
import pandas as pd

# Reshape wide claim data to long format

line_rows = []

for _, row in df_opps_claims.iterrows():  #df_claims.iterrows() goes through the DataFrame row by row
    n = row["line_ct"]
    for i in range(1, n + 1):
        line_rows.append({
            "claim_id": row["claim_id"],
            "bene_id": row["bene_id"],
            "prov_id": row["prov_id"],
            "line_ct": row["line_ct"],
            "line_no": i,
            "hcpcs": row.get(f"line{i}_hcpcs"),
            "rvcd": row.get(f"line{i}_rev"),
            "chrg": row.get(f"line{i}_chrg"),
            "units": row.get(f"line{i}_units"),
            "revdt": row.get(f"line{i}_revdt")
        })

df_lines = pd.DataFrame(line_rows)
df_lines["rvcd"] = df_lines["rvcd"].astype(int).astype(str).str.zfill(4)

In [45]:
df_lines.dtypes
df_lines.head(5)

,claim_id,bene_id,prov_id,line_ct,line_no,hcpcs,rvcd,chrg,units,revdt
0,0,2102,303,5,1,J2505,0420,1180.55,5.0,2025-01-15
1,0,2102,303,5,2,J2778,0610,194.96,8.0,2025-01-13
2,0,2102,303,5,3,J9206,0360,257.16,3.0,2025-01-31
3,0,2102,303,5,4,J9206,0456,1096.90,6.0,2025-01-26
4,0,2102,303,5,5,G0463,0450,491.15,6.0,2025-01-04


In [46]:
# for prov_id and ccr_XXXX columns, here’s how to reshape from wide to long
df_prov_ccr_long = pd.wide_to_long(df_prov_ccr, 
                          stubnames="ccr", 
                          i="prov_id", 
                          j="rvcd", 
                          sep="_", 
                          suffix="\\d+").reset_index()

# stubnames: the common prefix of your variable types (like ccr, ccr_desc)
# i: the identifier column(s), here it's prov_id
# j: the new column created from the suffixes in your original column names — in this case, rvcd
# sep: the character that separates prefix and suffix in the column names (_)
# suffix: the pattern for suffix (like numbers), '\\d+' means match digits

In [47]:
df_prov_ccr_long.head()

,prov_id,rvcd,ccr
0,100,450,0.5082
1,101,450,0.5238
2,102,450,0.6605
3,103,450,0.6762
4,104,450,0.4972


In [48]:
df_prov_ccr_long = df_prov_ccr.melt(id_vars="prov_id", 
                      var_name="rvcd", 
                      value_name="ccr")

df_prov_ccr_long["rvcd"] = df_prov_ccr_long["rvcd"].str.replace("ccr_", "")

In [49]:
df_prov_ccr_long.head()

,prov_id,rvcd,ccr
0,100,0450,0.5082
1,101,0450,0.5238
2,102,0450,0.6605
3,103,0450,0.6762
4,104,0450,0.4972


In [53]:
df_lines_w_ccr = df_lines.merge(df_prov_ccr_long, on=["prov_id", "rvcd"], how="left")

In [55]:
df_lines_w_cost = df_lines_w_ccr.copy()
df_lines_w_cost["line_cost"] = df_lines_w_cost["chrg"] * df_lines_w_cost["ccr"]
df_lines_w_cost.head()

,claim_id,bene_id,prov_id,line_ct,line_no,hcpcs,rvcd,chrg,units,revdt,ccr,line_cost
0,0,2102,303,5,1,J2505,0420,1180.55,5.0,2025-01-15,0.6359,750.711745
1,0,2102,303,5,2,J2778,0610,194.96,8.0,2025-01-13,0.4757,92.742472
2,0,2102,303,5,3,J9206,0360,257.16,3.0,2025-01-31,0.2010,51.689160
3,0,2102,303,5,4,J9206,0456,1096.90,6.0,2025-01-26,0.7762,851.413780
4,0,2102,303,5,5,G0463,0450,491.15,6.0,2025-01-04,0.2088,102.552120


## To calculate line-level cost and keep it in the claim-level (wide) format

In [ ]:
df_opps_claims = pd.read_csv(r"J:/Python/Learning/Data/opps_claims_simulated.csv")

In [56]:
df_claims_ccr = df_opps_claims.merge(df_prov_ccr, on="prov_id", how="left")
df_claims_ccr.head()

,claim_id,bene_id,prov_id,line_ct,line1_hcpcs,line1_rev,line1_chrg,line1_units,line1_revdt,line2_hcpcs,...,ccr_0420,ccr_0430,ccr_0440,ccr_0460,ccr_0480,ccr_0490,ccr_0520,ccr_0730,ccr_0610,ccr_0910
0,0,2102,303,5,J2505,420,1180.55,5,2025-01-15,J2778,...,0.6359,0.2379,0.7731,0.4555,0.2334,0.7224,0.9322,0.7951,0.4757,0.9234
1,1,2088,300,3,J3489,490,387.52,3,2025-01-19,J0885,...,0.1466,0.2288,0.7738,0.7820,0.2729,0.8111,0.1633,0.4131,0.4759,0.5279
2,2,2166,301,4,J0881,730,608.85,5,2025-01-26,G0463,...,0.4713,0.4230,0.9251,0.1636,0.6008,0.5274,0.8188,0.7476,0.7343,0.1323
3,3,2133,301,4,J1459,430,1412.27,4,2025-01-15,J0881,...,0.4713,0.4230,0.9251,0.1636,0.6008,0.5274,0.8188,0.7476,0.7343,0.1323
4,4,2088,300,1,J0885,250,58.01,3,2025-01-16,NaN,...,0.1466,0.2288,0.7738,0.7820,0.2729,0.8111,0.1633,0.4131,0.4759,0.5279


In [57]:
# Define cost calculator
def add_line_costs(row):
    for i in range(1, int(row["line_ct"]) + 1):
        rvcd = str(int(row[f"line{i}_rev"])).zfill(4)
        chrg = row[f"line{i}_chrg"]
        ccr = row.get(f"ccr_{rvcd}")
        
        if pd.notnull(chrg) and pd.notnull(ccr):
            cost = chrg * ccr
        else:
            cost = np.nan
        
        row[f"line{i}_cost"] = cost
    return row

In [58]:
df_claims_cost = df_claims_ccr.apply(add_line_costs, axis=1)

# axis=0 → applies the function column by column
# axis=1 → applies the function row by row

In [59]:
df_claims_cost.head()

,bene_id,ccr_0250,ccr_0260,ccr_0270,ccr_0300,ccr_0320,ccr_0360,ccr_0420,ccr_0430,ccr_0440,...,line4_revdt,line4_units,line5_chrg,line5_cost,line5_hcpcs,line5_rev,line5_revdt,line5_units,line_ct,prov_id
0,2102,0.9519,0.4578,0.2683,0.1806,0.4385,0.2010,0.6359,0.2379,0.7731,...,2025-01-26,6.0,491.15,102.55212,G0463,450.0,2025-01-04,6.0,5,303
1,2088,0.9246,0.8876,0.4171,0.8604,0.6190,0.1811,0.1466,0.2288,0.7738,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,300
2,2166,0.3030,0.3261,0.5738,0.8226,0.6880,0.5725,0.4713,0.4230,0.9251,...,2025-02-01,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4,301
3,2133,0.3030,0.3261,0.5738,0.8226,0.6880,0.5725,0.4713,0.4230,0.9251,...,2025-02-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4,301
4,2088,0.9246,0.8876,0.4171,0.8604,0.6190,0.1811,0.1466,0.2288,0.7738,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,300


In [64]:
df_claims_cost = df_claims_cost.drop(df_claims_cost.filter(like="ccr_").columns, axis=1)
df_claims_cost.head()

# df_claims.drop(df_claims.filter(regex="(^ccr_|cost)").columns, axis=1)
# ^ccr_ → starts with ccr_
# cost → anywhere in the column name
# axis=1 is column

,bene_id,claim_id,line1_chrg,line1_cost,line1_hcpcs,line1_rev,line1_revdt,line1_units,line2_chrg,line2_cost,...,line4_revdt,line4_units,line5_chrg,line5_cost,line5_hcpcs,line5_rev,line5_revdt,line5_units,line_ct,prov_id
0,2102,0,1180.55,750.711745,J2505,420,2025-01-15,5,194.96,92.742472,...,2025-01-26,6.0,491.15,102.55212,G0463,450.0,2025-01-04,6.0,5,303
1,2088,1,387.52,314.317472,J3489,490,2025-01-19,3,909.00,479.861100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,300
2,2166,2,608.85,455.176260,J0881,730,2025-01-26,5,399.49,52.852527,...,2025-02-01,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4,301
3,2133,3,1412.27,597.390210,J1459,430,2025-01-15,4,523.48,276.083352,...,2025-02-01,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4,301
4,2088,4,58.01,53.636046,J0885,250,2025-01-16,3,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,300


In [63]:
df_claims_cost.to_excel(r"J:/Python/Learning/Data/OPPS_claims_w_cost.xlsx", index=False)